# Project 2
### CS 5/7394 - Applied Machine Learning

- **Due** - March 11 @ 11:59 pm pushed to Github repo
- **Teams** - You can do this project solo or in pairs.  Not 3, not 4 not 5... Max of 2. If a 5394 student pairs with a 7394 student, the pair needs to do the 7394 work. 

Below are 6 Kaggle Datasets.  You will choose 1 to work with for this project. 

- [Airfare Prediction Dataset](https://www.kaggle.com/zwartfreak/airline-fare-prediction)
- [Chinese Rest Holiday Dataset](https://www.kaggle.com/holoong9291/chinese-rest-holiday-dataset-2020-to-2022)
- [Jigsaw Toxic Comment Classification Challenge](https://www.kaggle.com/julian3833/jigsaw-toxic-comment-classification-challenge?select=train.csv)
- [Latest Covid 19 Dataset Worldwide](https://www.kaggle.com/sandhyakrishnan02/latest-covid-19-dataset-worldwide)
- [Trains](https://www.kaggle.com/vahidehdashti/machinelearningdatabase-trains?select=trains-original.data)
- [Football Data top 5 Leagues](https://www.kaggle.com/sanjeetsinghnaik/football-data-top-5-leagues)

Merging disparate datasets is a staple of the data exploration process.  Therefore, for which ever data set above that you choose, you will need to independently find **an additional** dataset to merge with your selection.  The only requirement is that it add to the richness of the original dataset. Students in the 7000-level version of the class need to find two additional data sets to merge with the original selection. 

_Note_: If you want to start with a different data set, you need to get Fontenot's OK first.  

### Your Tasks

Below, there are cells that provide directions on what to do for the project.  

You can insert as many cells between the ones below as you'd like, but please **Do NOT** change the cells already provided. 

### Part 1 - Getting Started

- Import libraries
- Load original Data (which ever one you chose from the provided list) into a data frame. 
- Load your additional data set(s) into a data frame. 
- In a markdown cell, provide a brief description of your the data sets you've chosen to work with.  
- Develop a list of 3 - 4 questions that you hope to be able to answer after the exploration of the data and write them in this section. 

In [42]:
import numpy as np
import pandas as pd
%matplotlib inline

import scipy.stats as stats
from sklearn import metrics
from sklearn import preprocessing
from sklearn import  linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

import matplotlib.pyplot as plt
import seaborn as sn

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (30, 10),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}

sn.set_style('whitegrid')
sn.set_context('talk')

plt.rcParams.update(params)
pd.options.display.max_colwidth = 600


airfareTest = pd.read_csv('airfareTest.csv')
airfareTrain = pd.read_csv('airfareTrain.csv')
dieselPrices = pd.read_csv('diesel.csv')


#Todo
# Steps 3 and 4



### Clean Datasets 

In [43]:
#Standardiszing Attribute Names
airfareTest.rename(columns={'Airline':'airline',
                            'Date_of_Journey':'date',
                            'Source':'starting_city',
                            'Destination':'destination',
                            'Route':'route',
                            'Dep_Time':'depart_time',
                            'Arrival_Time':'arrival_time',
                            'Duration':'duration',
                            'Total_Stops':'total_stops',
                            'Additional Info':'misc'},inplace=True)


airfareTest = airfareTest.drop(['route','depart_time', 'arrival_time','total_stops','misc'], axis=1)

#Typecasting Attributes
    #AirFare
airfareTest['date'] = pd.to_datetime(airfareTest.date)
airfareTest['duration'] = pd.to_timedelta(airfareTest.duration).astype('timedelta64[h]')
airfareTest['duration'] = pd.to_numeric(airfareTest.duration)

airfareTest['airline'] = airfareTest.airline.astype('category')
airfareTest['starting_city'] = airfareTest.starting_city.astype('category')
airfareTest['destination'] = airfareTest.destination.astype('category')

    #Diesel
dieselPrices['date'] =  pd.to_datetime(dieselPrices.date)

### Part 2 - Data Inspection

Write some code to summarize the datasets.  Think about the following questions:
- What type of data is each variable? (think like a data scientist here, not a computer scientist)
- What is the total size of the data sets?
- What time boundaries are there in the dataset?  IOW, what time frame do they span?
- Are there any missing values in any of the variables? 

Do this with Intentionality.  Don't skimp. 

### What type of data is each variable?

In [44]:
print("=Airfare Types=\n",airfareTest.dtypes,"\n")
print("=Diesel Types=\n",dieselPrices.dtypes,"\n")

=Airfare Types=
 airline                category
date             datetime64[ns]
starting_city          category
destination            category
duration                float64
dtype: object 

=Diesel Types=
 city             object
date     datetime64[ns]
rate            float64
state            object
dtype: object 



### What is the total size of the data sets?

In [45]:
print("Shape of Airfare Test:  ", airfareTest.shape)
print("Shape of Diesel Prices: ",dieselPrices.shape)

Shape of Airfare Test:   (2671, 5)
Shape of Diesel Prices:  (4104, 4)


### What time boundaries are there in the dataset?  IOW, what time frame do they span?

In [46]:
print("Airfare start date: ",airfareTest['date'].min())
print("Airfare end date:   ",airfareTest['date'].max(),'\n')
print(pd.date_range(start=airfareTest['date'].min(), end=airfareTest['date'].max()))

print("\nDiesel start date: ",dieselPrices['date'].min())
print("Diesel end date:   ",dieselPrices['date'].max(),'\n')
print(pd.date_range(start=dieselPrices['date'].min(), end=dieselPrices['date'].max()))

Airfare start date:  2019-01-03 00:00:00
Airfare end date:    2019-12-06 00:00:00 

DatetimeIndex(['2019-01-03', '2019-01-04', '2019-01-05', '2019-01-06',
               '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10',
               '2019-01-11', '2019-01-12',
               ...
               '2019-11-27', '2019-11-28', '2019-11-29', '2019-11-30',
               '2019-12-01', '2019-12-02', '2019-12-03', '2019-12-04',
               '2019-12-05', '2019-12-06'],
              dtype='datetime64[ns]', length=338, freq='D')

Diesel start date:  2019-01-02 00:00:00
Diesel end date:    2019-12-31 00:00:00 

DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-05',
               '2019-01-06', '2019-01-07', '2019-01-08', '2019-01-09',
               '2019-01-10', '2019-01-11',
               ...
               '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25',
               '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29',
               '2019-12-30', '2019-12

### Are there any missing values in any of the variables? 

In [47]:
print("=Airline Values=\n")
print(
    "airline:\t\t",           airfareTest.airline.isnull().values.any(),
    "\ndate:\t\t\t",          airfareTest.date.isnull().values.any(),
    "\nstarting_city:\t\t",   airfareTest.starting_city.isnull().values.any(),
    "\ndestination:\t\t",     airfareTest.destination.isnull().values.any(),
    "\nduration:\t\t",        airfareTest.duration.isnull().values.any(),
)

print("\n=Diesel Values=\n")
print(
    "city:\t\t\t",            dieselPrices.city.isnull().values.any(),
    "\ndate:\t\t\t",          dieselPrices.date.isnull().values.any(),
    "\nrate:\t\t\t",          dieselPrices.rate.isnull().values.any(),
    "\nstate:\t\t\t",         dieselPrices.state.isnull().values.any(),

)

=Airline Values=



AttributeError: 'DataFrame' object has no attribute 'route'

### Part 3 - Data Description

- Create a data description (data dictionary) for your data sets.
    - Describe each variable
    - If categorical, what levels are present? If the levels are encoded, what do the codes mean?
    - If numeric, provide min, max, median and any other univariate stats you'd like to add in. 
- Where appropriate, provide histograms or other visualizations to characterize each variable. 

## Airline Fare Data Dictionarty

[Categorical]
- airline        : Name of airline carrier (Air India, IndiGo, Multiple Carriers, etc.)            
- starting_city  : Starting city (Delhi, Kolkata, Banglore, etc.)                             
- route          : Description of route between airports  (ex. BLR → BOM → DEL)
- depart_time    : Time the plane departed                   
- arrival_time   : Time the plane arrived at the destination 
- total_stops    : Total stops taken during the light        
- misc           : Additional information about the flight   

[Time]
- date           : Data                                      [Datetime]
- duration       : Total duration of the flight              [timedelta64]

In [ ]:
print('=Airline values=\n',airfareTest['airline'].value_counts())
print('\n=starting_city values=\n',airfareTest['starting_city'].value_counts())
print('\n=total_stops values=\n',airfareTest['total_stops'].value_counts())

In [ ]:
print("=Date=\nmin: ", airfareTest['date'].min() ,'\nmax:', airfareTest['date'].max(), '\nmedian: ', airfareTest['date'].median())
print("\n=Duration=\nmin: ", airfareTest['duration'].min() ,'\nmax:', airfareTest['duration'].max(), '\nmedian: ', airfareTest['duration'].median())

In [ ]:
sn.boxplot(data=airfareTest[['airline','duration']], x="airline", y="duration")



### Part 4 - Merge the data

Now that you have a better feel for each of your two (or three, for the 7394 students) data sets, it is time to merge them. Describe your strategy for merging the data sets and then actually perform the merge.  

Develop a strategy for verifying that the data is properly merged (hoping and finger-crossing are not valid strategies). 

### Part 5 - Explore Bivariate relationships

- Choose a reasoned set of variables to explore further.  You don't have to explore all possible pairs of variables, nor do we want to grade that much.  Choose 7 - 9 variables. One should be a variable that you'd like to predict (target variable) using the others (predictor variables). 
- List your predictor variables
- List your target variable
- Briefly describe why you have chosen these. 

Use any of the available visualizations from Seaborn to explore the relationships between the variables. Explore the relationships among the predictor variables as well as the relationship between each predictor variable and the target variable.  Which of the predictor variables are most strongly related?  Are there any interesting relationships between categorical predictors and numeric predictors?  If there are any dichotomous variables, does that influence any of the relationships? Are the relationships positive or negative? 

Below each plot, you should provide a description and interpretation of the plot.  Make sure to include why the variables in that plot were chosen and what you hope the reader would gain from it as well. 